In [1]:
import random
import string
import networkx as nx
from typing import Dict, Any, List, Tuple, Optional
import json
from collections import defaultdict

class NetworkGenerator:
    def __init__(self, schema_json: Dict[str, Any]):
        self.schema = schema_json
        self.node_instances: Dict[str, Dict[str, Dict[str, Any]]] = {}
        self.timestamp = 0
        self.operations = []
        
    def generate_random_value(self, feature_type: str) -> Any:
        """Generate random value based on feature type."""
        if feature_type == "string":
            return ''.join(random.choices(string.ascii_letters, k=8))
        elif feature_type == "float":
            return round(random.uniform(1, 1000), 2)
        elif feature_type == "integer":
            return random.randint(1, 100)
        else:
            return None

    def _get_reverse_topology(self) -> List[str]:
        """Get nodes in reverse topological order (leaf to root)."""
        graph = nx.DiGraph()
        
        # Add edges from the schema
        for edge_info in self.schema["edges"].values():
            graph.add_edge(edge_info["source"], edge_info["target"])
            
        # Return reverse topological sort (leaf to root)
        return list(reversed(list(nx.topological_sort(graph))))

    def _get_parent_type(self, node_type: str) -> Optional[str]:
        """Get the parent node type for a given node type."""
        for edge_info in self.schema["edges"].values():
            if edge_info["target"] == node_type:
                return edge_info["source"]
        return None

    def _create_node(self, node_type: str, parent_id: Optional[str] = None) -> str:
        """Create a single node with given type and optional parent ID."""
        # Generate node ID based on parent
        if parent_id is None:
            node_id = "1"
        else:
            # Get the last child number for this parent and increment
            existing_children = [n for n in self.node_instances.get(node_type, {}).keys() 
                               if n.startswith(f"{parent_id}-")]
            child_num = len(existing_children) + 1
            node_id = f"{parent_id}-{child_num}"

        # Generate node properties
        properties = {}
        for feature_name, feature_type in self.schema["nodes"][node_type]["features"].items():
            if feature_name == "id":
                properties[feature_name] = node_id
            else:
                properties[feature_name] = self.generate_random_value(feature_type)

        # Create node operation
        operation = {
            "action": "create",
            "type": "schema",
            "payload": {
                "node_id": node_id,
                "node_type": node_type,
                "properties": properties
            },
            "timestamp": self.timestamp
        }
        self.timestamp += 1
        self.operations.append(operation)
        
        # print("Node created:", node_id)
        
        # Store node instance
        if node_type not in self.node_instances:
            self.node_instances[node_type] = {}
        self.node_instances[node_type][node_id] = operation
        
        # Create edge to parent if exists
        if parent_id is not None:
            parent_type = self._get_parent_type(node_type)
            edge_type = next(
                edge_name for edge_name, edge_info in self.schema["edges"].items()
                if edge_info["source"] == parent_type and edge_info["target"] == node_type
            )
            
            edge_operation = self._create_edge(parent_id, node_id, edge_type)
            self.operations.append(edge_operation)
            
        return node_id

    def _create_edge(self, source_id: str, target_id: str, edge_type: str) -> Dict[str, Any]:
        """Create an edge between two nodes."""
        edge_schema = self.schema["edges"][edge_type]["features"]
        properties = {
            feature_name: self.generate_random_value(feature_type)
            for feature_name, feature_type in edge_schema.items()
        }
        
        operation = {
            "action": "create",
            "type": "schema",
            "payload": {
                "source_id": source_id,
                "target_id": target_id,
                "edge_type": edge_type,
                "properties": properties
            },
            "timestamp": self.timestamp
        }
        self.timestamp += 1
        return operation

    def _create_subtree(self, node_type: str, count: int, parent_id: Optional[str] = None) -> List[str]:
        """Recursively create a subtree starting from the given node type."""
        node_ids = []
        for i in range(count):
            # Create current node
            node_id = self._create_node(node_type, parent_id)
            node_ids.append(node_id)
            
            # Get child type if exists
            child_type = None
            for edge_info in self.schema["edges"].values():
                if edge_info["source"] == node_type:
                    child_type = edge_info["target"]
                    break
            
            # Recursively create children if any
            if child_type:
                child_count = self.nodes_per_type.get(child_type, 2)
                self._create_subtree(child_type, child_count, node_id)
                
        return node_ids

    def create_network(self, nodes_per_type: Dict[str, int]) -> List[Dict[str, Any]]:
        """Generate network starting from leaf nodes."""
        self.nodes_per_type = nodes_per_type
        self.operations = []
        self.node_instances = {}
        
        # Get nodes in reverse topological order
        topology = self._get_reverse_topology()
        
        # Start with the root nodes (last in reverse topology)
        root_type = topology[-1]
        root_count = nodes_per_type.get(root_type, 2)
        
        # Create the network starting from root
        self._create_subtree(root_type, root_count)
        
        return self.operations

def generate_test_network(schema_file_path: str, nodes_per_type: Dict[str, int]) -> List[Dict[str, Any]]:
    """Utility function to generate test network from schema file."""
    with open(schema_file_path, 'r') as f:
        schema = json.load(f)
    
    generator = NetworkGenerator(schema)
    return generator.create_network(nodes_per_type)

In [ ]:
# Example schema is provided as a dictionary
schema = json.load(open("../metadata/relations.json", "r"))
generator = NetworkGenerator(schema)

custom_counts = {
    "BusinessUnit": 1,
    "ProductFamily": 3,
    "ProductOffering": 6,
    "Facility": 3,
    "Parts": 8,
    "Warehouse": 3,
    "Supplier": 3
}
create_ops = generator.create_network(custom_counts)

print(f"Generated {len(create_ops)} operations")

In [3]:
API_URL = "http://localhost:8000"

timestamp = 0
version = "v3"

In [ ]:
import requests
import time

for op in create_ops:
    op["version"] = version
    requests.post(f"{API_URL}/schema/live/update", json=op)
    timestamp += 1
    time.sleep(0.4)